In [1]:
import requests
import urllib
from bs4 import BeautifulSoup
import re
import io
import gzip
import time
import csv
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import glob
import os
from os import path
import datetime
from dateutil.parser import parse
from selenium.webdriver.support.ui import Select


In [2]:
def get_date(d,m,y, driver):
#     print(d, m, y)
    yy = 2021
    if (int(d) < 10):
        if (len(str(d)) < 2):
            d = "0" + str(d)
#     print(d, m, y)
    while (int(y) < yy):
        yy -= 10
        year = driver.find_element_by_xpath('/html/body/div[2]/div/div/select[2]')
        segment = Select(year)
        segment.select_by_visible_text(str(yy))
    year = driver.find_element_by_xpath('/html/body/div[2]/div/div/select[2]')
    segment = Select(year)
    segment.select_by_visible_text(y)

    month = driver.find_element_by_xpath('/html/body/div[2]/div/div/select[1]')
    segment = Select(month)
    segment.select_by_visible_text(m)
#     time.sleep(5)
    days = driver.find_element_by_xpath("//table/tbody/tr/td/a[text()="+str(d)+"]")
    days.click()
    
        

In [3]:
def traverse_stock(security, driver):
    if (path.exists("C:\\Users\\venu\\Desktop\\Stock Market Analysis\\Data\\Corporate Actions\\Corporate_Actions_" + security + ".csv")):
#         print("present")
        df = pd.read_csv("C:\\Users\\venu\\Desktop\\Stock Market Analysis\\Data\\Corporate Actions\\Corporate_Actions_" + security + ".csv")
        fr_df = df.tail(1)['Ex Date']
        fr_date = fr_df.iloc[0]
        f = parse(fr_date)
        fd = f.date()
        if (fd == (datetime.datetime.today() - datetime.timedelta(days=1)).date()):
            return None
        else:
#             print("not current date")
            months = {1 : "Jan", 2 : "Feb", 3 : "Mar", 4 : "Apr", 5 : "May", 6 : "Jun", 
                     7 : "Jul", 8 : "Aug", 9 : "Sep", 10 : "Oct", 11 : "Nov", 12 : "Dec"}
            day = 1 if (fd.day + 1 == 32) else fd.day + 1
            month = -1
            if (fd.day + 1 == 32):
                month = fd.month + 1
                if (month == 13):
                    month = 1
            else:
                month = fd.month
            year = -1
            if (fd.day + 1 == 32 and fd.month + 1 == 13):
                year = fd.year + 1
            else:
                year = fd.year
            element = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_SmartSearch_smartSearch"]')
            element.clear()
            element.send_keys(security)
            element.send_keys(Keys.ENTER)
            time.sleep(2)
            from_date = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_txtDate"]')
            from_date.clear()
            from_date.click()
            get_date(str(day),months[month],str(year), driver)
            time.sleep(2)
            to_date = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_txtTodate"]')
            to_date.clear()
            to_date.click()
            cd = datetime.datetime.today()
            get_date(str(cd.day),months[cd.month],str(cd.year), driver)
            time.sleep(2)
            submit = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_btnSubmit"]')
            submit.click()
            time.sleep(2)
            store_stock(security, driver)
            return None


#     print("Intitial non presence")
    element = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_SmartSearch_smartSearch"]')
    element.clear()
    element.send_keys(security)
    element.send_keys(Keys.ENTER)
    time.sleep(2)

#     print("Not present")
    months = {1 : "Jan", 2 : "Feb", 3 : "Mar", 4 : "Apr", 5 : "May", 6 : "Jun", 
             7 : "Jul", 8 : "Aug", 9 : "Sep", 10 : "Oct", 11 : "Nov", 12 : "Dec"}
    from_date = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_txtDate"]')
    from_date.clear()
    from_date.click()
    get_date("01","Aug","2007", driver)
    to_date = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_txtTodate"]')
    to_date.clear()
    to_date.click()
    cd = datetime.datetime.today()
    get_date(str(cd.day),months[cd.month],str(cd.year), driver)
#     get_date("28", "Feb", "2021", driver)
#     print("submit click")
    submit = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_btnSubmit"]')
    submit.click()
    time.sleep(5)
    store_stock(security, driver)

In [4]:
def store_stock(security, driver):
    try :
#         print("downloading")
        download_stock = driver.find_element_by_xpath("/html/body/div[1]/form/div[4]/div/div/div[2]/div/div/div[2]/a/i")
        download_stock.click()
        time.sleep(8)
        remake_stock(security)
    except Exception as e:
#         print(e)
        pass

In [5]:
def remake_stock(security):
    files_list = os.listdir("C:\\Users\\venu\\Desktop\\Stock Market Analysis\\Data\\Corporate Actions\\")
    if ("Corporate_Actions_" + security +  ".csv") not in files_list:
#         print("not present")
        path = "C:\\Users\\venu\\Desktop\\Stock Market Analysis\\Data\\Corporate Actions\\Corporate_Actions" + ".csv"
        stock = pd.read_csv(path)
        stock.to_csv("C:\\Users\\venu\\Desktop\\Stock Market Analysis\\Data\\Corporate Actions\\Corporate_Actions_" + security + ".csv", index=False)
        os.remove(path)
    else:
#         print("already present")
        path = "C:\\Users\\venu\\Desktop\\Stock Market Analysis\\Corporate Actions\\Corporate_Actions.csv"
        merge_files = []
        for file in files_list:
            if file.startswith("Corporate_Actions"):
                merge_files.append(file)
#         print(merge_files)
        pth = "C:\\Users\\venu\\Desktop\\Stock Market Analysis\\Data\\Corporate Actions\\"
        combined_csv = pd.concat([pd.read_csv(pth + merge_files[f]) for f in range(len(merge_files) - 1, -1, -1)])
        combined_csv.to_csv( pth + "Corporate_Actions_" + security + ".csv", index=False, encoding='utf-8-sig')
        os.remove(path + "Corporate_Actions.csv")

In [6]:
def download_corporate_actions(nums):
    chromeOptions = webdriver.ChromeOptions()
#     chromeOptions.add_argument("--headless")
    chromeOptions.add_experimental_option("prefs", {"download.default_directory": "C:\\Users\\venu\\Desktop\\Stock Market Analysis\\Data\\Corporate Actions"})
    driver = webdriver.Chrome(ChromeDriverManager().install(),chrome_options=chromeOptions)
    url = "https://www.bseindia.com/corporates/corporate_act.aspx"
    driver.get(url)
    traverse_stock(str(nums), driver)
    driver.quit()


In [7]:
# download_corporate_actions(532210)

In [8]:
df_equity = pd.read_csv("C:\\Users\\venu\\Desktop\\Stock Market Analysis\\Data\\Equity.csv")
security_numbers = df_equity["Security Code"].tolist()

In [9]:
# security_numbers[282 : 299]

In [10]:
# %%time

# for sec_num in security_numbers[282 : 299]:
#     print(sec_num)
#     download_corporate_actions(sec_num)

In [11]:
sec_numb = [500285, 500298, 500317, 500319, 500356, 500358, 500388, 500400, 500402, 500405, 500414, 500456, 500458, 500890, 501110, 501144, 501242, 501261, 501421, 501477]

In [12]:
%%time

for sec_num in sec_numb[8 : ]:
    print(sec_num)
    download_corporate_actions(sec_num)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


500402




[WDM] - Driver [C:\Users\venu\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache
<ipython-input-6-bf128eaa9cfb>:5: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(ChromeDriverManager().install(),chrome_options=chromeOptions)
[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


500405




[WDM] - Driver [C:\Users\venu\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache
[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


...

501421




[WDM] - Driver [C:\Users\venu\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache
[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


501477




[WDM] - Driver [C:\Users\venu\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


Wall time: 7min 3s
